<a href="https://colab.research.google.com/github/Marcin19721205/IntroductionToBigDataWSB/blob/main/Zadanie_Nr4_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analiza koszykowa
Przeanalizuj przykład zamieszczony w pliku analiza_koszykowa_01a_s.r . Następnie przeprowadź w
języku R analizę koszykową za pomocą algorytmu Apriori dla transakcji zakupowych wczytanych z
podanego niżej pliku.  

zadanie w Python

Wczytaj transakcje z pliku ItemList.csv . Uwaga: Plik z transakcjami nie jest klasycznym
plikiem w formacie CSV i należy go wczytać następująco

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

#wczytaj dane
transactions_list = []
with open('sample_data/ItemList.csv', 'r') as file:
    for line in file:
        # Split by comma, strip whitespace from each item, and filter out empty strings
        items = [item.strip() for item in line.strip().split(',') if item.strip()]
        if items: # Only add non-empty transactions
            transactions_list.append(items)

# DataFrame
df = pd.DataFrame({'items': transactions_list})
#
#podgląd
print("Pierwsze 5 transakcji:")
for i, trans in enumerate(transactions_list[:5]):
    print(f"Transakcja {i+1}: {trans}")

print("\nInformacje o danych (DataFrame z listą przedmiotów):")
print(df.head())
print(df.info())

Pierwsze 5 transakcji:
Transakcja 1: ['itemList']
Transakcja 2: ['1', 'sausage', 'whole milk', 'semi-finished bread', 'yogurt']
Transakcja 3: ['2', 'whole milk', 'pastry', 'salty snack']
Transakcja 4: ['3', 'canned beer', 'misc. beverages']
Transakcja 5: ['4', 'sausage', 'hygiene articles']

Informacje o danych (DataFrame z listą przedmiotów):
                                               items
0                                         [itemList]
1  [1, sausage, whole milk, semi-finished bread, ...
2               [2, whole milk, pastry, salty snack]
3                  [3, canned beer, misc. beverages]
4                     [4, sausage, hygiene articles]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14964 entries, 0 to 14963
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   items   14964 non-null  object
dtypes: object(1)
memory usage: 117.0+ KB
None


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


#punkt b

Za pomocą algorytmu Apriori wylicz reguły wynikowe dla minimalnego wsparcia = 0.01 oraz
minimalnej pewności = 0.01 . Wylicz tylko reguły długości 2 (argument minlen=2) tzn.
zawierające przynajmniej 2 produkty.

In [3]:
#Analiza asocjacji (Apriori)
import re, warnings
warnings.filterwarnings("ignore", message=r".*datetime\.utcnow\(\) is deprecated.*")
#
# Wczytanie pliku "basket" (1 wiersz = 1 koszyk; 1. kolumna = ID) ----
def read_basket_file(path: Path):
    tx = []
    with open(path, 'r', encoding='utf-8') as f:
        for ln, raw in enumerate(f):
            line = raw.strip()
            if not line:
                continue
            items = [x.strip() for x in line.split(',') if x.strip()]

            # pomiń nagłówek typu 'itemList'
            if ln == 0 and len(items) == 1 and items[0].lower() in {"itemlist","items"}:
                continue
            # usuń ID transakcji (1. token liczbowy)
            if items and re.fullmatch(r"\d+", items[0]):
                items = items[1:]

            # deduplikacja w koszyku (jak rm.duplicates=TRUE)
            seen = set()
            deduped = [x for x in items if not (x in seen or seen.add(x))]
            if deduped:
                tx.append(deduped)
    return tx

#Ścieżka do pliku
basket_path = Path("sample_data/ItemList.csv")   # podmień, jeśli masz inną lokalizację
transactions = read_basket_file(basket_path)

print(f"Transakcji: {len(transactions)} | Przykładowe: {transactions[:2]}")

#One-hot
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_tx = pd.DataFrame(te_ary, columns=te.columns_)   # wiersz=koszyk, kolumna=produkt, True/False

#Zbiory częste (Apriori; sup ≥ 0.01
freq = apriori(df_tx, min_support=0.01, use_colnames=True)
freq = freq.sort_values(['support','itemsets'], ascending=[False, True]).reset_index(drop=True)
print("\nTOP zbiory częste (support):")
print(freq.head(10))

#Reguły A→B (conf ≥ 0.01), + minlen=2 (co najmniej 1 w LHS i 1 w RHS
rules = association_rules(freq, metric="confidence", min_threshold=0.01)
rules = rules[(rules['antecedents'].apply(len) >= 1) & (rules['consequents'].apply(len) >= 1)]

# czytelne kolumny
rules_disp = rules.copy()
rules_disp['antecedents'] = rules_disp['antecedents'].apply(lambda s: ', '.join(sorted(list(s))))
rules_disp['consequents'] = rules_disp['consequents'].apply(lambda s: ', '.join(sorted(list(s))))
rules_disp = rules_disp[['antecedents','consequents','support','confidence','lift']]

print(f"\nReguł (minlen=2): {len(rules_disp)}")
print("\nTOP wg support:")
print(rules_disp.sort_values('support', ascending=False).head(20))

print("\nTOP wg confidence:")
print(rules_disp.sort_values('confidence', ascending=False).head(20))

print("\nTOP wg lift:")
print(rules_disp.sort_values('lift', ascending=False).head(20))

#
#

fig = px.scatter(
    rules_disp,
    x='support', y='confidence',
    color='lift', size='lift',
    hover_data=['antecedents','consequents'],
    title='Reguły: support vs confidence (kolor = lift)'
)
fig.show()

# filtr reguł „sensownych”
rules_good = rules_disp.query('lift > 1')

fig2 = px.scatter(
    rules_good, x='support', y='confidence',
    color='lift', size='lift',
    hover_data=['antecedents','consequents'],
    title='Reguły (lift > 1): support vs confidence'
)
# zapis do PNG/HTML (w Colab wymaga kaleido)
# pip install -q kaleido
# fig2.write_image('rules_scatter.png', scale=2)
fig2.write_html('rules_scatter.html')  # zawsze działa



Transakcji: 14963 | Przykładowe: [['sausage', 'whole milk', 'semi-finished bread', 'yogurt'], ['whole milk', 'pastry', 'salty snack']]

TOP zbiory częste (support):
    support            itemsets
0  0.157923        (whole milk)
1  0.122101  (other vegetables)
2  0.110005        (rolls/buns)
3  0.097106              (soda)
4  0.085879            (yogurt)
5  0.069572   (root vegetables)
6  0.067767    (tropical fruit)
7  0.060683     (bottled water)
8  0.060349           (sausage)
9  0.053131      (citrus fruit)

Reguł (minlen=2): 10

TOP wg support:
        antecedents       consequents   support  confidence      lift
0        whole milk  other vegetables  0.014837    0.093948  0.769430
1  other vegetables        whole milk  0.014837    0.121511  0.769430
2        rolls/buns        whole milk  0.013968    0.126974  0.804028
3        whole milk        rolls/buns  0.013968    0.088447  0.804028
4              soda        whole milk  0.011629    0.119752  0.758296
5        whole milk     